In [1]:
import pandas_gbq
import pandas as pd

def proper_sampling_analysis():
    """Use proper sampling to get representative data"""

    # Stratified sampling query
    query = """
    SELECT *
    FROM (
      SELECT *,
        ROW_NUMBER() OVER (PARTITION BY city ORDER BY RAND()) as rn
      FROM `gulf-retail-30days.gulf_retail.customers`
    )
    WHERE rn <= 40
    """

    df = pandas_gbq.read_gbq(query, project_id="gulf-retail-30days")

    print("🎯 PROPERLY SAMPLED CUSTOMERS DATA")
    print("=" * 50)
    print(f"Total records: {len(df)}")
    print(f"City distribution:")
    print(df['city'].value_counts())
    print(f"\nDate range: {df['registration_date'].min()} to {df['registration_date'].max()}")

    return df

# Run the analysis
df = proper_sampling_analysis()

Downloading: 100%|██████████|
🎯 PROPERLY SAMPLED CUSTOMERS DATA
Total records: 200
City distribution:
city
Abu Dhabi    40
Doha         40
Dubai        40
Riyadh       40
Sharjah      40
Name: count, dtype: int64

Date range: 2019-01-15 to 2023-02-03


In [2]:
import pandas_gbq
import pandas as pd
import numpy as np

def comprehensive_quality_check():
    """Run comprehensive data quality checks on all tables"""

    project_id = "gulf-retail-30days"

    print("🔍 COMPREHENSIVE DATA QUALITY ASSESSMENT")
    print("=" * 60)

    # Check each table systematically
    tables = ['customers', 'orders', 'order_lines', 'products']

    for table in tables:
        print(f"\n📊 TABLE: {table.upper()}")
        print("-" * 40)

        # Sample data with proper randomization
        query = f"""
        SELECT *
        FROM `gulf-retail-30days.gulf_retail.{table}`
        ORDER BY RAND()
        LIMIT 1000
        """

        df = pandas_gbq.read_gbq(query, project_id=project_id)

        # 1. Basic Info
        print(f"📈 Shape: {df.shape}")
        print(f"📝 Columns: {list(df.columns)}")

        # 2. Data Types
        print(f"🔧 Data Types:")
        df_dtypes = df.dtypes
        print(df_dtypes)

        # 3. Missing Values Analysis
        print(f"❓ Missing Values:")
        missing_count = df.isnull().sum()
        missing_pct = (missing_count / len(df)) * 100

        for col in df.columns:
            if missing_count[col] > 0:
                print(f"   {col}: {missing_count[col]} ({missing_pct[col]:.1f}%)")
            else:
                print(f"   {col}: ✅ No missing values")

        # 4. Unique Values & Cardinality
        print(f"🎯 Unique Values:")
        for col in df.columns:
            unique_count = df[col].nunique()
            print(f"   {col}: {unique_count} unique values")
            if unique_count <= 10:  # Show values for low-cardinality columns
                print(f"      Values: {df[col].unique()}")

        # 5. Statistical Summary for Numeric Columns
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        if len(numeric_cols) > 0:
            print(f"📊 Numeric Columns Summary:")
            print(df[numeric_cols].describe())

        # 6. Date Column Analysis (if any)
        date_cols = df.select_dtypes(include=['datetime64']).columns
        for col in date_cols:
            print(f"📅 {col} Date Range: {df[col].min()} to {df[col].max()}")

        # 7. Sample Data Preview
        print(f"👀 First 5 rows:")
        print(df.head())

        print("\n" + "="*40)


def check_business_rules():
    """Check data against business rules"""

    print("\n💼 BUSINESS RULES VALIDATION")
    print("=" * 50)

    # Rule 1: Order dates should not be in future
    query = """
    SELECT
        COUNT(*) as future_orders,
        MAX(order_date) as latest_order_date
    FROM `gulf-retail-30days.gulf_retail.orders`
    WHERE order_date > CURRENT_DATE()
    """
    result = pandas_gbq.read_gbq(query, project_id="gulf-retail-30days")
    print(f"📅 Future orders: {result['future_orders'].iloc[0]} (should be 0)")

    # Rule 2: Line amounts should be positive
    query = """
    SELECT
        COUNT(*) as negative_amounts
    FROM `gulf-retail-30days.gulf_retail.order_lines`
    WHERE net_line_amount < 0
    """
    result = pandas_gbq.read_gbq(query, project_id="gulf-retail-30days")
    print(f"💰 Negative line amounts: {result['negative_amounts'].iloc[0]} (should be 0)")

    # Rule 3: Quantities should be positive
    query = """
    SELECT
        COUNT(*) as negative_quantities
    FROM `gulf-retail-30days.gulf_retail.order_lines`
    WHERE qty < 0
    """
    result = pandas_gbq.read_gbq(query, project_id="gulf-retail-30days")
    print(f"📦 Negative quantities: {result['negative_quantities'].iloc[0]} (should be 0)")

# Run comprehensive checks
if __name__ == "__main__":
    comprehensive_quality_check()
    check_business_rules()

    print("\n✅ DATA QUALITY ASSESSMENT COMPLETE")
    print("📋 Next: Review findings and plan data cleaning strategy")

🔍 COMPREHENSIVE DATA QUALITY ASSESSMENT

📊 TABLE: CUSTOMERS
----------------------------------------
Downloading: 100%|██████████|
📈 Shape: (1000, 4)
📝 Columns: ['customer_id', 'first_name', 'city', 'registration_date']
🔧 Data Types:
customer_id           Int64
first_name           object
city                 object
registration_date    dbdate
dtype: object
❓ Missing Values:
   customer_id: ✅ No missing values
   first_name: ✅ No missing values
   city: ✅ No missing values
   registration_date: ✅ No missing values
🎯 Unique Values:
   customer_id: 1000 unique values
   first_name: 5 unique values
      Values: ['Khalid' 'Fatima' 'Sara' 'Omar' 'Ahmed']
   city: 5 unique values
      Values: ['Sharjah' 'Abu Dhabi' 'Riyadh' 'Doha' 'Dubai']
   registration_date: 885 unique values
📊 Numeric Columns Summary:
       customer_id
count       1000.0
mean       1108.82
std     579.946841
min          102.0
25%         606.75
50%         1113.5
75%        1624.25
max         2101.0
👀 First 5 rows:


In [3]:
# @title AI prompt cell

import ipywidgets as widgets
from IPython.display import display, HTML, Markdown,clear_output
from google.colab import ai

dropdown = widgets.Dropdown(
    options=[],
    layout={'width': 'auto'}
)

def update_model_list(new_options):
    dropdown.options = new_options
update_model_list(ai.list_models())

text_input = widgets.Textarea(
    placeholder='Ask me anything....',
    layout={'width': 'auto', 'height': '100px'},
)

button = widgets.Button(
    description='Submit Text',
    disabled=False,
    tooltip='Click to submit the text',
    icon='check'
)

output_area = widgets.Output(
     layout={'width': 'auto', 'max_height': '300px','overflow_y': 'scroll'}
)

def on_button_clicked(b):
    with output_area:
        output_area.clear_output(wait=False)
        accumulated_content = ""
        for new_chunk in ai.generate_text(prompt=text_input.value, model_name=dropdown.value, stream=True):
            if new_chunk is None:
                continue
            accumulated_content += new_chunk
            clear_output(wait=True)
            display(Markdown(accumulated_content))

button.on_click(on_button_clicked)
vbox = widgets.GridBox([dropdown, text_input, button, output_area])

display(HTML("""
<style>
.widget-dropdown select {
    font-size: 18px;
    font-family: "Arial", sans-serif;
}
.widget-textarea textarea {
    font-size: 18px;
    font-family: "Arial", sans-serif;
}
</style>
"""))
display(vbox)


GridBox(children=(Dropdown(layout=Layout(width='auto'), options=('google/gemini-2.5-flash', 'google/gemini-2.5…